## Preparation

In [1]:
import numpy as np
from sympy import *
from sympy.solvers import solve
init_printing(use_unicode=True)

def num_sym(p: int) -> int:
    return int(p * (p + 1) / 2)

def solve_lyapunov_cov(M: Matrix, C: Matrix, p: int) -> Matrix:
    Sigma = MatrixSymbol("Sigma", p, p).as_explicit()
    Sigma_solved = solve(M*Sigma + Sigma*M.T + C, Sigma)
    scheme = [[Sigma_solved[Sigma[i,j]] for i in range(p)] for j in range(p)]
    Sigma_new = Matrix(scheme)
    return Sigma_new

def create_A_Sigma(cov: Matrix, p: int) -> Matrix:
    row_num = num_sym(p=p)
    col_num = int(p * p)
    A_Sigma = MatrixSymbol("A", row_num, col_num).as_mutable()
    for l in range(p):
        for k in range(l+1):
            for i in range(p):
                for j in range(p):
                    if (j != k) & (j != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 0
                    elif (j == k) & (k != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[l, i]
                    elif (j == l) & (l != k):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[k, i]
                    elif (j == k) & (j == l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 2 * cov[j, i]
    return A_Sigma

def create_A(A_Sigma: Matrix, mean: Matrix, p: int) -> Matrix:
    identity = mean[0] * eye(p)
    for i in range(1, p):
        identity = Matrix.hstack(identity, mean[i] * eye(p))
    return Matrix.vstack(A_Sigma, identity)

def create_c(vech_C: Matrix, b: float, index: int, p: int) -> Matrix:
    unit_vector = eye(p).col(index)
    return Matrix.vstack(vech_C, - b * unit_vector)

## Test 1
Estimate 1000 true matrices $M^*$, then create the matrix $A$ and vector $c$, calculate $\text{det}(A)$ and if $\text{det}(A)\neq 0$ estimate $\hat{M}$ and calculate the Frobenius distance to the true $M^*$.

In [ ]:
p = 3
num_eq = num_sym(p=p) + p
num_seed = 1000
d = 0.9 # Probability of edges in graph

m11, m12, m13 = symbols('m11'), symbols('m12'), symbols('m13')
m21, m22, m23 = symbols('m21'), symbols('m22'), symbols('m23')
m31, m32, m33 = symbols('m31'), symbols('m32'), symbols('m33')
M = Matrix([[m11, m12, m13], [m21, m22, m23], [m31, m32, m33]])

C = 2 * eye(p) # volatility matrix

vec_M = Matrix([[M[j, i]] for i in range(p) for j in range(p)]) # j faster than i
vech_C = Matrix([[C[i, j]] for i in range(p) for j in range(i, p)])

# Intervention
index_intervention = 1
b = 2
unit_v = eye(p).col(index_intervention)

estimates_3nodes_test1 = np.empty(shape=(num_seed), dtype=object)
statistics_3nodes_test1 = np.empty(shape=(num_seed, 3), dtype=float)

for seed in range(1, num_seed + 1):
    np.random.seed(seed=seed)

    bernoulli_matrix = np.random.binomial(1, d, (p, p))
    normal_matrix = np.random.normal(0, 1, (p, p))

    temp_M = bernoulli_matrix * normal_matrix
    for i in range(p): # adjust diagonal entries s.t. M stable
        row_sum = np.sum(np.abs(temp_M[i, :])) - np.abs(temp_M[i, i])
        temp_M[i, i] = - row_sum - np.abs(normal_matrix[i,i])

    concrete_M = Matrix(temp_M)

    cov = solve_lyapunov_cov(M=concrete_M, C=C, p=p)
    mean = b * concrete_M.inv() * unit_v

    A_Sigma = create_A_Sigma(cov=cov, p=p)
    A = create_A(A_Sigma=A_Sigma, mean=mean, p=p)

    c = create_c(vech_C=vech_C, b=b, index=index_intervention, p=p)

    det_A = det(A)
    solution = solve(A * vec_M + c, vec_M)

    if solution:
        M_est = M.subs(solve(A * vec_M + c, vec_M))
        M_est_np = matrix2numpy(M_est).astype(float)
        M_true = matrix2numpy(concrete_M).astype(float)
        frob = np.linalg.norm(M_est_np - M_true)

        estimates_3nodes_test1[seed - 1] = M_est_np.round(3)
        statistics_3nodes_test1[seed - 1, 0] = det_A
        statistics_3nodes_test1[seed - 1, 1] = frob
        statistics_3nodes_test1[seed - 1, 2] = 1
    # elif det_A == 0: # det(A) = 0
        # print("Solution exists at seed ", seed, ": ", solution, ", but det(A)=0.")
        # statistics_3nodes_test1[seed - 1, 0] = det_A
        # statistics_3nodes_test1[seed - 1, 1] = 0.0
        # statistics_3nodes_test1[seed - 1, 2] = 0
    else: # empty solution
        statistics_3nodes_test1[seed - 1, 0] = det_A
        statistics_3nodes_test1[seed - 1, 1] = 0.0
        statistics_3nodes_test1[seed - 1, 2] = 0

Had Problem at seed = 220, so investigate what caused error.

In [ ]:
np.random.seed(seed=220)

bernoulli_matrix = np.random.binomial(1, d, (p, p))
normal_matrix = np.random.normal(0, 1, (p, p))

temp_M = bernoulli_matrix * normal_matrix
for i in range(p): # adjust diagonal entries s.t. M stable
    row_sum = np.sum(np.abs(temp_M[i, :])) - np.abs(temp_M[i, i])
    temp_M[i, i] = - row_sum - np.abs(normal_matrix[i,i])

concrete_M = Matrix(temp_M)

cov = solve_lyapunov_cov(M=concrete_M, C=C, p=p)
mean = b * concrete_M.inv() * unit_v

A_Sigma = create_A_Sigma(cov=cov, p=p)
A = create_A(A_Sigma=A_Sigma, mean=mean, p=p)

c = create_c(vech_C=vech_C, b=b, index=index_intervention, p=p)

det_A = det(A)
print(det_A)
print(A * vec_M + c)

M_est = M.subs(solve(A * vec_M + c, vec_M))
# M_est_np = matrix2numpy(M_est).astype(float)
M_true = matrix2numpy(concrete_M).astype(float)
# frob = np.linalg.norm(M_est_np - M_true)
M_est

In [ ]:
concrete_M

In [ ]:
cov

In [ ]:
A

In [ ]:
det_A

In [ ]:
A * vec_M + c